In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357274 sha256=f5414301bff2fb784fa51f8e04cc4d360ff7a94d1f9377c47b166d81ed78b31d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate

In [3]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [15]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [36]:
raw_ratings = data.raw_ratings

In [37]:
df = pd.DataFrame(raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])

In [38]:
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [35]:
print(f"\nКількість користувачів: {df['user'].nunique()}")
print(f"Кількість фільмів: {df['item'].nunique()}")


Кількість користувачів: 943
Кількість фільмів: 1682


In [39]:
def evaluate_model(model, trainset, testset):
    model.fit(trainset)
    predictions = model.test(testset)
    return predictions

def calculate_metrics(predictions):
    from surprise import accuracy
    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)
    return rmse, mae

In [41]:
param_grid = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}

### Модель SVD і крос-валідація для підбору параметрів

In [42]:
print("Оптимізація SVD...")
gs_svd = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs_svd.fit(data)

best_svd = gs_svd.best_estimator['rmse']
predictions_svd = evaluate_model(best_svd, trainset, testset)
rmse_svd, mae_svd = calculate_metrics(predictions_svd)


Оптимізація SVD...
RMSE: 0.9604
MAE:  0.7682


### Модель SVD++

In [44]:
print("\nОптимізація SVD++...")
gs_svdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)
gs_svdpp.fit(data)

best_svdpp = gs_svdpp.best_estimator['rmse']
predictions_svdpp = evaluate_model(best_svdpp, trainset, testset)
rmse_svdpp, mae_svdpp = calculate_metrics(predictions_svdpp)

print(f"Найкращі параметри SVD++: {gs_svdpp.best_params['rmse']}")
print(f"RMSE: {rmse_svdpp:.4f}, MAE: {mae_svdpp:.4f}")



Оптимізація SVD++...
RMSE: 0.9603
MAE:  0.7681
Найкращі параметри SVD++: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}
RMSE: 0.9603, MAE: 0.7681


### Модель NMF 

In [56]:
# Визначення параметрів для пошуку
param_grid_nmf = {
    'n_epochs': [50, 100],
    'n_factors': [15, 20, 25],
    'reg_pu': [0.06, 0.1],
    'reg_qi': [0.06, 0.1]
}

In [57]:
print("\nОптимізація NMF...")
gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=5)
gs_nmf.fit(data)

best_nmf = gs_nmf.best_estimator['rmse']
predictions_nmf = evaluate_model(best_nmf, trainset, testset)
rmse_nmf, mae_nmf = calculate_metrics(predictions_nmf)

print(f"Найкращі параметри NMF: {gs_nmf.best_params['rmse']}")
print(f"RMSE: {rmse_nmf:.4f}, MAE: {mae_nmf:.4f}")


Оптимізація NMF...
RMSE: 0.9375
MAE:  0.7431
Найкращі параметри NMF: {'n_epochs': 100, 'n_factors': 25, 'reg_pu': 0.1, 'reg_qi': 0.1}
RMSE: 0.9375, MAE: 0.7431


### Вибір найкращої моделі


In [58]:
best_model = min([(rmse_svd, 'SVD'), (rmse_svdpp, 'SVD++'), (rmse_nmf, 'NMF')], key=lambda x: x[0])

print(f"\nНайкраща модель: {best_model[1]}")


Найкраща модель: NMF
